<a href="https://colab.research.google.com/github/paulIruaRosero/Paul-Irua-Rosero-Se-ales-y-Sistemas/blob/main/Ejercicios/Untitled16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install matplotlib-venn
!apt-get -qq install -y libfluidsynth1
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
!apt-get -qq install -y graphviz && pip install pydot
!pip install cartopy
!pip install pydub
!pip install wget


E: Package 'libfluidsynth1' has no installation candidate
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=238f0d5bc6c098136332d701c4770917a92bf93726f91c5bcbcfd5032ec8921e
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [26]:
import libarchive
import pydot
import cartopy

#------------------------------------------------------------------------------
# ID del archivo en Google Drive
FILEID="1ULmbFz5jLR4-3KZnf2YbKbXuXN7c9Pb7"
FILENAME="clave2.wav"  # Nombre del archivo de salida

# Descargar el archivo desde Google Drive y guardarlo como audioClaveSeñal.wav
!wget --no-check-certificate "https://docs.google.com/uc?export=download&id=$FILEID" -O $FILENAME

# Verificar si el archivo se descargó correctamente
!ls -lh $FILENAME
#------------------------------------------------------------------------------

#------------------------------------------------------------------------------
import soundfile as sf
# Cargar el archivo de audio descargado
file_path = "clave2.wav"  # Ruta del archivo
audio_data, sample_rate = sf.read(file_path)

# Mostrar información básica del audio
print(f"Frecuencia de muestreo: {sample_rate} Hz")
print(f"Número de muestras: {len(audio_data)}")
print(f"Forma del audio: {audio_data.shape}")

# Si el audio tiene dos canales, convertirlo a mono
if len(audio_data.shape) == 2:
    audio_data = audio_data.mean(axis=1)  # Promediar los canales estéreo para obtener mono
    print("Audio convertido a mono.")

# Verificar duración del audio en segundos
duracion = len(audio_data) / sample_rate
print(f"Duración del audio: {duracion:.2f} segundos")
#------------------------------------------------------------------------------

#------------------------------------------------------------------------------
import os
import numpy as np
import librosa
import librosa.display
import soundfile as sf

def segment_audio_by_energy(audio_path, output_folder, frame_length=1024, hop_length=256, padding=0.05, merge_gap=0.3):
    # Cargar el audio asegurando una frecuencia de muestreo fija
    y, sr = librosa.load(audio_path, sr=16000, mono=True)  # Convertimos a 16 kHz y mono

    # Calcular la energía de la señal
    energy = np.array([
        np.sum(np.square(y[i : i + frame_length]))
        for i in range(0, len(y), hop_length)
    ])

    # Normalizar la energía
    energy = energy / np.max(energy)

    # Establecer un umbral dinámico basado en el percentil 75 en lugar de un valor fijo
    energy_threshold = np.percentile(energy, 55)

    # Encontrar regiones con energía mayor al umbral
    voiced_frames = energy > energy_threshold

    # Determinar los índices de los segmentos hablados
    change_points = np.diff(voiced_frames.astype(int))
    start_indices = np.where(change_points == 1)[0] * hop_length
    end_indices = np.where(change_points == -1)[0] * hop_length

    if len(end_indices) == 0 or (len(start_indices) > 0 and start_indices[0] > end_indices[0]):
        end_indices = np.append(end_indices, len(y))
    if len(start_indices) == 0 or (len(end_indices) > 0 and end_indices[-1] < start_indices[-1]):
        start_indices = np.insert(start_indices, 0, 0)

    # Filtrar segmentos pequeños (ruido)
    min_duration = 0.1 * sr  # Reducimos a 100 ms para evitar cortes de palabras
    valid_segments = [(start, end) for start, end in zip(start_indices, end_indices) if end - start > min_duration]

    # Unir segmentos muy cercanos (para evitar cortes en sílabas)
    merged_segments = []
    prev_start, prev_end = valid_segments[0]
    for start, end in valid_segments[1:]:
        if start - prev_end < merge_gap * sr:  # Si la pausa es menor que 300 ms, unir segmentos
            prev_end = end
        else:
            merged_segments.append((prev_start, prev_end))
            prev_start, prev_end = start, end
    merged_segments.append((prev_start, prev_end))  # Agregar el último segmento

    # Aplicar padding antes y después de cada segmento
    padding_samples = int(padding * sr)
    merged_segments = [(max(0, start - padding_samples), min(len(y), end + padding_samples)) for start, end in merged_segments]

    # Crear la carpeta de salida si no existe
    os.makedirs(output_folder, exist_ok=True)

    # Guardar cada segmento como un archivo separado
    segment_links = []
    for i, (start, end) in enumerate(merged_segments):
        segment = y[start:end]
        output_file = os.path.join(output_folder, f'segment_{i+1}.wav')
        sf.write(output_file, segment, sr)
        print(f'Segmento {i+1} guardado: {output_file}')
        segment_links.append((f"audioClave{i+1}", output_file, 1))  # Nombre, ruta, etiqueta

    return segment_links  # Retornar la lista con los datos

# Uso del código
input_audio = "clave2.wav"
output_directory = "segmented_audios"
segments_data = segment_audio_by_energy(input_audio, output_directory)
print (output_directory)
#------------------------------------------------------------------------------

#------------------------------------------------------------------------------
# ID del archivo en Google Drive
FILEID="1kKOid1g9DjCbQ3OBf1Zu0PuW-ErayIBU"
FILENAME="noclave.wav"  # Nombre del archivo de salida

# Descargar el archivo desde Google Drive y guardarlo como audioClaveSeñal.wav
!wget --no-check-certificate "https://docs.google.com/uc?export=download&id=$FILEID" -O $FILENAME

# Verificar si el archivo se descargó correctamente
!ls -lh $FILENAME
#------------------------------------------------------------------------------

#------------------------------------------------------------------------------
import soundfile as sf  # Librería para leer archivos de audio

# Cargar el archivo de audio descargado
file_path = "noclave.wav"  # Ruta del archivo
audio_data, sample_rate = sf.read(file_path)

# Mostrar información básica del audio
print(f"Frecuencia de muestreo: {sample_rate} Hz")
print(f"Número de muestras: {len(audio_data)}")
print(f"Forma del audio: {audio_data.shape}")

# Si el audio tiene dos canales, convertirlo a mono
if len(audio_data.shape) == 2:
    audio_data = audio_data.mean(axis=1)  # Promediar los canales estéreo para obtener mono
    print("Audio convertido a mono.")

# Verificar duración del audio en segundos
duracion = len(audio_data) / sample_rate
print(f"Duración del audio: {duracion:.2f} segundos")

input_audio = "noclave.wav"
output_directory = "segmented_audiosError"
segment=segment_audio_by_energy(input_audio, output_directory)

#------------------------------------------------------------------------------

#------------------------------------------------------------------------------

import os
import numpy as np
import librosa
import scipy.fftpack as fft
import pickle
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Función para calcular el espectro de Fourier
def compute_spectrum(signal, sr, n_fft=2064):
    spectrum = np.abs(fft.fft(signal, n=n_fft))  # Magnitud del espectro
    max_val = np.max(spectrum)
    if max_val > 0:
     spectrum = spectrum / max_val  # Normalizar evitando división por cero
    else:
     spectrum = np.zeros_like(spectrum)  # Si es 0, evitar división errónea
    return spectrum[:n_fft // 2]      # Usamos solo la mitad (simetría de la FFT)
    #spectrum = spectrum / np.min(spectrum)  # Normalizar por el valor minimo
    #return spectrum[:n_fft // 2]  # Usamos solo la mitad (simetría de la FFT)

# Función para procesar una carpeta de audios y crear la base de datos
def create_database(folder_path, sr=22050):
    database = []
    filenames = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".wav"):  # Solo procesar archivos .wav
            file_path = os.path.join(folder_path, filename)
            signal, _ = librosa.load(file_path, sr=sr)  # Cargar el audio
            spectrum = compute_spectrum(signal, sr)  # Calcular el espectro
            database.append(spectrum)  # Agregar a la base de datos
            filenames.append(filename)  # Guardar el nombre del archivo
    return np.array(database), filenames

# Rutas de las carpetas en Colab
correct_folder = "/content/segmented_audios"  # Carpeta con la palabra correcta
error_folder = "/content/segmented_audiosError"  # Carpeta con palabras incorrectas

# Crear la carpeta 'error_folder' si no existe
if not os.path.exists(error_folder):
    os.makedirs(error_folder)
    print(f"Carpeta '{error_folder}' creada.")

# Crear las bases de datos
correct_database, correct_filenames = create_database(correct_folder)  # Base de datos para la palabra correcta
error_database, error_filenames = create_database(error_folder)  # Base de datos para palabras incorrectas

# Combinar las bases de datos y crear etiquetas
if error_database.size == 0: # Check if error_database is empty
    X = correct_database  # If empty, use only correct_database
    labels = np.array([1] * len(correct_database))  # Labels for correct_database only
    filenames = correct_filenames  # Filenames for correct_database only
else:
    X = np.vstack((correct_database, error_database))  # Combinar espectros
    labels = np.array([1] * len(correct_database) + [0] * len(error_database))  # Etiquetas (1: correcta, 0: incorrecta)
    filenames = correct_filenames + error_filenames  # Nombres de los archivos

#------------------------------------------------------------------------------

#------------------------------------------------------------------------------
# Reducción a 2D usando t-SNE o PCA
fmax = 7000  # Considerar solo frecuencias hasta fmax Hz
red_ = TSNE(perplexity=20, n_components=2, random_state=123, learning_rate='auto', init='pca')
# red_ = PCA(n_components=2, random_state=123)  # Alternativa: PCA
X_2D = red_.fit_transform(X[:, :fmax])  # Reducir a 2D
#------------------------------------------------------------------------------

#------------------------------------------------------------------------------
# Guardar las bases de datos en archivos
with open("correct_database.pkl", "wb") as f:
    pickle.dump(correct_database, f)

with open("error_database.pkl", "wb") as f:
    pickle.dump(error_database, f)

print("Bases de datos creadas y guardadas correctamente.")
print(f"Tamaño de la base de datos palabra clave correcta: {len(correct_database)}")
print(f"Tamaño de la base de datos de palabra no clave: {len(error_database)}")
#------------------------------------------------------------------------------

#------------------------------------------------------------------------------
import pickle
from pydub import AudioSegment
from scipy.spatial.distance import euclidean
from google.colab import files

# Función para convertir un archivo de audio a WAV
def convert_to_wav(input_file, output_file):
    audio = AudioSegment.from_file(input_file)
    audio.export(output_file, format="wav")
    print(f"Archivo convertido a WAV: {output_file}")

# Función para calcular el espectro de Fourier
def compute_spectrum(signal, sr, n_fft=1024):
    spectrum = np.abs(fft.fft(signal, n=n_fft))  # Magnitud del espectro
    spectrum = spectrum[:n_fft // 2]  # Usamos solo la mitad (simetría de la FFT)
    spectrum = spectrum / np.max(spectrum)  # Normalizar el espectro
    return spectrum

# Función para calcular el índice del valor mínimo en el espectro
def compute_min_spectrum_index(signal, sr, n_fft=1024):
    spectrum = compute_spectrum(signal, sr, n_fft)
    min_index = np.argmin(spectrum)  # Índice del valor mínimo en el espectro
    return min_index

# Función para cortar el audio basado en la energía (igual que en tu código)
def segment_audio_by_energy(y, sr, frame_length=1024, hop_length=256, padding=0.05, merge_gap=0.3):
    # Calcular la energía de la señal
    energy = np.array([
        np.sum(np.square(y[i : i + frame_length]))
        for i in range(0, len(y), hop_length)
    ])

    # Normalizar la energía
    energy = energy / np.max(energy)

    # Establecer un umbral dinámico basado en el percentil 55
    energy_threshold = np.percentile(energy, 55)

    # Encontrar regiones con energía mayor al umbral
    voiced_frames = energy > energy_threshold

    # Determinar los índices de los segmentos hablados
    change_points = np.diff(voiced_frames.astype(int))
    start_indices = np.where(change_points == 1)[0] * hop_length
    end_indices = np.where(change_points == -1)[0] * hop_length

    if len(end_indices) == 0 or (len(start_indices) > 0 and start_indices[0] > end_indices[0]):
        end_indices = np.append(end_indices, len(y))
    if len(start_indices) == 0 or (len(end_indices) > 0 and end_indices[-1] < start_indices[-1]):
        start_indices = np.insert(start_indices, 0, 0)

    # Filtrar segmentos pequeños (ruido)
    min_duration = 0.1 * sr  # Reducimos a 100 ms para evitar cortes de palabras
    valid_segments = [(start, end) for start, end in zip(start_indices, end_indices) if end - start > min_duration]

    # Unir segmentos muy cercanos (para evitar cortes en sílabas)
    merged_segments = []
    prev_start, prev_end = valid_segments[0]
    for start, end in valid_segments[1:]:
        if start - prev_end < merge_gap * sr:  # Si la pausa es menor que 300 ms, unir segmentos
            prev_end = end
        else:
            merged_segments.append((prev_start, prev_end))
            prev_start, prev_end = start, end
    merged_segments.append((prev_start, prev_end))  # Agregar el último segmento

    # Aplicar padding antes y después de cada segmento
    padding_samples = int(padding * sr)
    merged_segments = [(max(0, start - padding_samples), min(len(y), end + padding_samples)) for start, end in merged_segments]

    return merged_segments

# Función para comparar un índice mínimo con una base de datos usando distancia euclidiana
def compare_with_database(input_min_index, database):
    min_distance = float('inf')
    for db_spectrum in database:  # Iterate over spectra in the database
        db_min_index = np.argmin(db_spectrum)  # Get the index of the minimum value in the spectrum
        distance = euclidean([input_min_index], [db_min_index])  # Calculate distance using the minimum indices
        if distance < min_distance:
            min_distance = distance
    return min_distance

# Subir un archivo de audio

uploaded = files.upload()
uploaded_file = list(uploaded.keys())[0]

# Convertir a WAV si no está en formato WAV
if not uploaded_file.endswith(".wav"):
    output_file = "converted_audio.wav"
    convert_to_wav(uploaded_file, output_file)
    audio_file = output_file
else:
    audio_file = uploaded_file

# Cargar el archivo de audio
signal, sr = librosa.load(audio_file, sr=16000, mono=True)  # Convertimos a 16 kHz y mono

# Cortar el audio basado en la energía (usando la misma lógica que en la base de datos)
segments = segment_audio_by_energy(signal, sr)

# Si hay segmentos, tomar el primero (asumimos que es el más relevante)
if len(segments) > 0:
    start, end = segments[0]
    trimmed_signal = signal[start:end]
else:
    trimmed_signal = signal  # Si no se encuentra ningún segmento, usar la señal completa

# Calcular el índice del valor mínimo en el espectro del audio subido
input_min_index = compute_min_spectrum_index(trimmed_signal, sr)

# Cargar las bases de datos
with open("correct_database.pkl", "rb") as f:
    correct_database = pickle.load(f)

with open("error_database.pkl", "rb") as f:
    error_database = pickle.load(f)

# Comparar con la base de datos de palabras correctas
distance_correct = compare_with_database(input_min_index, correct_database)

# Comparar con la base de datos de palabras incorrectas
distance_error = compare_with_database(input_min_index, error_database)

# Tomar una decisión
if distance_correct < distance_error:
    print("Acceso permitido, el audio ingresado pertenece a la palabra clave.")
else:
    print("Acceso denegado, el audio ingresado no pertenece a la palabra clave.")

--2025-02-22 04:11:08--  https://docs.google.com/uc?export=download&id=1ULmbFz5jLR4-3KZnf2YbKbXuXN7c9Pb7
Resolving docs.google.com (docs.google.com)... 142.250.65.110, 2607:f8b0:4025:804::200e
Connecting to docs.google.com (docs.google.com)|142.250.65.110|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1ULmbFz5jLR4-3KZnf2YbKbXuXN7c9Pb7&export=download [following]
--2025-02-22 04:11:08--  https://drive.usercontent.google.com/download?id=1ULmbFz5jLR4-3KZnf2YbKbXuXN7c9Pb7&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 172.217.0.65, 2607:f8b0:4025:810::2001
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|172.217.0.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4876366 (4.7M) [audio/wav]
Saving to: ‘clave2.wav’

clave2.wav          100%[===================>]   4.65M  --.-KB/s    in 0.1s    

2025-02-22 04:11:10 

Saving Palabra No-Clave.wav to Palabra No-Clave (11).wav
Acceso denegado, el audio ingresado no pertenece a la palabra clave.
